In [1]:
# How to use Pretrained models with PyTorch
# Simple Classifier using resnet50
# code by GunhoChoi

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.models as models
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.autograd import Variable

batch_size = 3
learning_rate =0.0002
epoch = 50

resnet = models.resnet50(pretrained=True)

In [2]:
# Input pipeline from a folder containing multiple folders of images
# we can check the classes, class_to_idx, and filename with idx

img_dir = "./images"
img_data = dset.ImageFolder(img_dir, transforms.Compose([
            transforms.RandomSizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            ]))

print(img_data.classes)
print(img_data.class_to_idx)
print(img_data.imgs)

['dog', 'onepun']
{'onepun': 1, 'dog': 0}
[('dog/25.png', 0), ('dog/282.png', 0), ('dog/221.png', 0), ('dog/74.png', 0), ('onepun/1482.jpg', 1), ('onepun/1450.jpg', 1), ('onepun/1490.jpg', 1), ('onepun/1446.jpg', 1), ('onepun/1454.jpg', 1)]


In [3]:
# After we get the list of images, we can turn the list into batches of images
# with torch.utils.data.DataLoader()

img_batch = data.DataLoader(img_data, batch_size=batch_size,
                            shuffle=True, num_workers=2)

for img,label in img_batch:
    print(img.size())
    print(label)

torch.Size([3, 3, 224, 224])

 1
 1
 0
[torch.LongTensor of size 3]

torch.Size([3, 3, 224, 224])

 0
 0
 0
[torch.LongTensor of size 3]

torch.Size([3, 3, 224, 224])

 1
 1
 1
[torch.LongTensor of size 3]



In [4]:
# test of the result coming from resnet model

img = Variable(img)
print(resnet(img))

Variable containing:
 2.1611  0.9679  0.5624  ...   0.5942  1.0807  0.3515
-2.4065 -0.0536 -1.2708  ...  -1.1309  1.4694  1.4215
-2.9848 -1.6954 -0.7385  ...  -0.7834  0.8400  0.9059
[torch.FloatTensor of size 3x1000]



In [5]:
# we have 2 categorical variables so 1000 -> 500 -> 2
# test the whole process

model = nn.Sequential(
            nn.Linear(1000,500),
            nn.ReLU(),
            nn.BatchNorm1d(500),
            nn.Linear(500,2),
            nn.ReLU()
            )

print(model(resnet(img)))

Variable containing:
 0.3056  0.2306
 0.0000  0.0000
 0.1929  0.0000
[torch.FloatTensor of size 3x2]



In [6]:
# define loss func & optimizer

loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [7]:
# In order to train with GPU, we need to put the model and variables
# by doing .cuda()

resnet.cuda()
model.cuda()

for i in range(epoch):
    for img,label in img_batch:
        img = Variable(img).cuda()
        label = Variable(label).cuda()

        optimizer.zero_grad()
        output = model(resnet(img))
        loss = loss_func(output,label)
        loss.backward()
        optimizer.step()

    if i % 10 ==0:
        print(loss)

Variable containing:
 0.6594
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 0.3142
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 0.1387
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 0.7628
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 0.1347
[torch.cuda.FloatTensor of size 1 (GPU 0)]



In [8]:
# Check Accuracy of the trained model
# Need to get used to using .cuda() and .data 

model.eval()
correct = 0
total = 0

for img,label in img_batch:
    img = Variable(img).cuda()
    label = Variable(label).cuda()
    
    output = model(resnet(img))
    _, pred = torch.max(output.data,1)
    
    total += label.size(0)
    correct += (pred == label.data).sum()   

print("Accuracy: {}".format(correct/total))

Accuracy: 0.8888888888888888
